# Abstract

This project implemented machine learning methods to differentiate between music by different composers. This was achieved by inspection of MIDI metadata and audio features (predominantly consisting of spectral analysis).

# Dataset Description

This project was based on a public set of classical compositions for piano. The dataset was sourced from http://www.piano-midi.de/. MIDI files were taken as the raw data and **jAudio** (http://jaudio.sourceforge.net/) was used to extract features from the MIDI. There are 127 MIDI files (datapoints) in the dataset. The size of the dataset was increased by splitting the MIDI files into 16 second samples before extracting the audio features.

## Attributes

The chosen target variable was composer name

The attributes/features for the data are as follows:

1. MFCC - The Mel-frequency Cepstral Coefficients. (extracted using **jAudio**)
2. Spectral Flux - a measure of how quickly the power spectrum of a signal is changing. (extracted using **jAudio**)
3. Compactness (extracted using **jAudio**)
4. Spectral Variability (extracted using **jAudio**)
5. Root Mean Square - a measure of the average amplitude of the signal. (extracted using **jAudio**)
6. Zero Crossings (extracted using **jAudio**)
7. Strongest Frequency Via Zero Crossings (extracted using **jAudio**)
8. Strongest Frequency Via Spectral Centroid (extracted using **jAudio**)
9. Strongest Frequency Via FFT Maximum (extracted using **jAudio**)
10. LPC - Linear Predictive Coding  (extracted using **jAudio**)
11. Method of Moments (extracted using **jAudio**)
12. Partial Based Spectral Centroid (extracted using **jAudio**)
13. Partial Based Spectral Flux (extracted using **jAudio**)
14. Peak Based Spectral Smoothness (extracted using **jAudio**)
15. Relative Difference Function (extracted using **jAudio**)
16. Area Method of Moments (extracted using **jAudio**)
17. Area Method of Moments of MFCCs (extracted using **jAudio**)
18. Key Signature (extracted directly from MIDI)
19. Time Signature (extracted directly from MIDI)
20. Average Tempo (extracted directly from MIDI)

The last 3 attributes were only used used in the **Discrete Na&iuml;ve Bayes** algorithm.

## Data Structuring and Normalization

The data has been limited to only include the works of **Chopin**, **Mozart**, **Schubert**, and **Beethoven**. This choice was made to narrow the problem space. The data was preprocessed by passing it through **jAudio** to extract the desired features.

## Splitting the data

# Algorithms

## Gaussian Na&iuml;ve Bayes 

This algorithm used the audio features we extracted with **jAudio**.

### Implementation Details

The implementation of this algorithm assumed a normal distribution for each feature in the data. The variances and means of these distributions was learned from the training data and then used to calculate the likelihoods for the test data. When performing a classification with this algorithm, the probability of generating a given feature value within a given class needs to be calculated. Since the probability of generating any given feature value in continuous data is zero, the algorithm instead looks at the probability of generating a value within $10^{-9} \sigma^2$ of the given feature value. This interval seems sufficiently tight to reject false positives but wide enough to mitigate the problem of finding a probability of 0 for all values.

### Error On Test Set


## Discrete Na&iuml;ve Bayes

This algorithm used the metadata and message data directly from the MIDI files to predict the composer of a given piece. The reason for this implementation was to try and perform the classification with discrete data straight from the MIDI files in order to avoid inaccuracies and complications introduced in the processing of the continuous; multi-dimensional features we extracted using jAudio. Although it is not a particularly interesting way to view the data (since the composer of a piece is usually specified in the metadata of a MIDI file) ~it may provide an interesting contrast to the other method in terms of results.~

### Implementation Details

A straightforward implementation of Na&iuml;ve Bayes with Laplace smoothing. The features used in this algorithm were extracted directly from the MIDI files and were chosen for simplicity's sake. Due to inconsistent labelling of composer names in the MIDI files, some datapoints got lost in the data preparation process and thus were not used for this algorithm. The chosen features for this algorithm are as follows:

#### Key Signature

The first key signature given in the MIDI file. Subsequent key signature changes were ignored for simplicity. Key signatures are represented as strings such as `C` or `Ab`. Due to inconsistencies in the representation of key MIDI metadata, some key signatures were represented twice in two different ways (for example **F<sup>#</sup>** and **G<sup>b</sup>** which are in fact the same key).

![Key Frequencies](report/plots/key_freq.png)

#### Time Signature

The first time signature given in the MIDI file. Subsequent time signature changes were ignored since they are fairly uncommon in the dataset, and would simply complicate the problem. Time signatures are represented as strings such as `3,4` or `5,4`.

#### Mean Tempo

An average of the tempo throughout the whole piece. This is measured in ticks. The mean tempo was then discretized by finding the mean $\mu$ and variance $\sigma^2$ of the mean tempos across all data points and then turning them into discrete values according to the rule:

$T_{class} = 0 \quad$ if $T_{value} < \mu - \sigma^2$, Low tempo.

$T_{class} = 1 \quad$ if $\mu - \sigma^2 <= T_{value} <= \mu + \sigma^2$, Mid tempo.

$T_{class} = 2 \quad$ if $T_{value} < \mu + \sigma^2$, High tempo.

High mean tempos were absent in the dataset.

### Error On Test Set

When the data was split randomly into training and testing data, one of the composers tended to be over-represented in the training data, leading to the model favouring that composer in the prediction. This can be clearly seen in the confusion matrix below.

|| Mozart  | Beethoven |
|-| -- | -- |
|**Mozart**| 10  | 7  |
|**Beethoven**| 0  | 2  |

The problem got worse when all four composers were present in the data since Chopin is hugely overrepresented.

||Chopin|Schubert| Mozart  | Beethoven |
|-| -- | -- | -- | -- |
|**Chopin**| 21 | 14 | 9 | 6 |
|**Schubert**| 0 | 0 | 0 | 0 |
|**Mozart**| 0 | 0 | 0 | 0 |
|**Beethoven**| 0 | 0 | 0 | 0 |

This reinforces the decision to ensure that all composers are equally represented in the training data. When this decision was implemented, it improved the results of this algorithm for the two composers case but didn't improve the results for the four composers case.

|| Mozart  | Beethoven |
|-| -- | -- |
|**Mozart**| 9  | 9  |
|**Beethoven**| 1  | 4  |

||Chopin|Schubert| Mozart  | Beethoven |
|-| -- | -- | -- | -- |
|**Chopin**| 30 | 14 | 9 | 14 |
|**Schubert**| 0 | 0 | 0 | 0 |
|**Mozart**| 0 | 0 | 0 | 0 |
|**Beethoven**| 0 | 0 | 0 | 0 |

The lack of improvement in the four composers case serves to demonstrate that these features are not sufficient to characterise a composer's style and thus are a poor choice compared to the extracted audio features used in the other algorithms.


## Logistic Regression

This algorithm used the audio features we extracted with **jAudio**.

### Implementation Details

### Error On Test Set

### Hyperparameters

# Discussion of Results

## Best Possible Performance

## Recommendations to Others Working on This Data